# Предсказываем стоимость мед страховки

## Загружаем необходимые библиотеки

In [3]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

## Загружаем данные

Собраны данные:
- age: возраст
- sex: пол
- bmi: индекс массы тела
- children: количество детей, охваченных медицинским страхованием / количество иждивенцев
- smoker: курение
- region: регион (northeast, southeast, southwest, northwest).
- charges: индивидуальные медицинские расходы (его и хотим предсказать)

In [4]:
# Загружаем данные из файла insurance.csv в переменную df
df = pd.read_csv('insurance.csv', sep=',')

In [5]:
# Посмотрим первые 5 строк
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


## Смотрим статистику, что нет пропусков и отсуствующих значений

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


In [7]:
# Посмотрим статистику
df.describe()

,age,bmi,children,charges
count,1338.000000,1338.000000,1338.000000,1338.000000
mean,39.207025,30.663397,1.094918,13270.422265
std,14.049960,6.098187,1.205493,12110.011237
min,18.000000,15.960000,0.000000,1121.873900
25%,27.000000,26.296250,0.000000,4740.287150
50%,39.000000,30.400000,1.000000,9382.033000
75%,51.000000,34.693750,2.000000,16639.912515
max,64.000000,53.130000,5.000000,63770.428010


## Преобразуем строковые данные

In [8]:
# Заменяем пол и курение на числа
df['sex']=df['sex'].map({'male':1, 'female':0})
df['smoker']=df['smoker'].map({'yes':1,'no':0})

In [9]:
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,0,27.900,0,1,southwest,16884.92400
1,18,1,33.770,1,0,southeast,1725.55230
2,28,1,33.000,3,0,southeast,4449.46200
3,33,1,22.705,0,0,northwest,21984.47061
4,32,1,28.880,0,0,northwest,3866.85520


In [10]:
# Заменяем регион на набор отдельных колонок (region_southwest, region_southeast и тд)
df = pd.get_dummies(df, columns=['region'])
df.head()

,age,sex,bmi,children,smoker,charges,region_northeast,region_northwest,region_southeast,region_southwest
0,19,0,27.900,0,1,16884.92400,False,False,False,True
1,18,1,33.770,1,0,1725.55230,False,False,True,False
2,28,1,33.000,3,0,4449.46200,False,False,True,False
3,33,1,22.705,0,0,21984.47061,False,True,False,False
4,32,1,28.880,0,0,3866.85520,False,True,False,False


## Формируем признаки и целевую переменную

In [11]:
# Сформируем признаки
X = df[['age', 'sex',	'bmi',	'children',	'smoker',	'region_northeast',	'region_northwest',	'region_southeast',	'region_southwest']]
X

,age,sex,bmi,children,smoker,region_northeast,region_northwest,region_southeast,region_southwest
0,19,0,27.900,0,1,False,False,False,True
1,18,1,33.770,1,0,False,False,True,False
2,28,1,33.000,3,0,False,False,True,False
3,33,1,22.705,0,0,False,True,False,False
4,32,1,28.880,0,0,False,True,False,False
...,...,...,...,...,...,...,...,...,...
1333,50,1,30.970,3,0,False,True,False,False
1334,18,0,31.920,0,0,True,False,False,False
1335,18,0,36.850,0,0,False,False,True,False
1336,21,0,25.800,0,0,False,False,False,True


In [12]:
# Сформируем целевую переменную
y = df[['charges']]
y

,charges
0,16884.92400
1,1725.55230
2,4449.46200
3,21984.47061
4,3866.85520
...,...
1333,10600.54830
1334,2205.98080
1335,1629.83350
1336,2007.94500


## Разделяем данные на тестовую выборку и для обучения

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [14]:
X_train

,age,sex,bmi,children,smoker,region_northeast,region_northwest,region_southeast,region_southwest
1171,43,0,26.700,2,1,False,False,False,True
1134,28,1,33.820,0,0,False,True,False,False
270,18,1,29.370,1,0,False,False,True,False
849,55,1,32.775,0,0,False,True,False,False
358,23,1,41.910,0,0,False,False,True,False
...,...,...,...,...,...,...,...,...,...
1246,45,0,25.700,3,0,False,False,False,True
785,35,0,27.700,3,0,False,False,False,True
1237,58,0,28.215,0,0,False,True,False,False
617,49,1,25.600,2,1,False,False,False,True


In [15]:
X_test

,age,sex,bmi,children,smoker,region_northeast,region_northwest,region_southeast,region_southwest
0,19,0,27.900,0,1,False,False,False,True
692,20,1,32.395,1,0,False,True,False,False
474,54,1,25.100,3,1,False,False,False,True
914,33,1,24.605,2,0,False,True,False,False
654,59,0,35.200,0,0,False,False,True,False
...,...,...,...,...,...,...,...,...,...
1192,58,0,32.395,1,0,True,False,False,False
189,29,0,32.110,2,0,False,True,False,False
61,25,1,33.660,4,0,False,False,True,False
687,40,1,41.690,0,0,False,False,True,False


## Создаем и обучаем модель линейной регресии

In [16]:
# Создаем и обучаем модель линейной регресии
lr = LinearRegression()
lr.fit(X_train, y_train)

,fit_intercept,True
,copy_X,True
,tol,1e-06
,n_jobs,None
,positive,False


In [17]:
# Посмотрим коэффициенты
lr.coef_

array([[  255.16110028,  -178.95403975,   386.36011062,   472.1598432 ,
        23660.48451687,   796.31478827,   103.5435397 ,  -712.0724384 ,
         -187.78588957]])

## Получаем предсказание и оцениваем качество

In [18]:
# Получаем предсказание
lr.predict(X_test)

array([[25279.51073537],
       [ 4195.41124872],
       [34365.86642529],
       [ 4974.92013384],
       [14121.61248839],
       [ 2420.58640743],
       [33115.32389457],
       [12416.68281085],
       [29666.34906509],
       [14738.11855385],
       [12172.86453469],
       [  878.99587799],
       [ 9199.82348347],
       [29569.33356604],
       [ 8762.62144649],
       [12090.30561393],
       [ 7432.64409108],
       [16384.26876974],
       [ 7294.09033278],
       [14864.6622239 ],
       [30785.1136271 ],
       [10162.58868078],
       [12502.69863273],
       [-1907.44181712],
       [36739.2198252 ],
       [ 9204.62480679],
       [10329.43634347],
       [-1341.7206177 ],
       [10884.25253969],
       [ 8067.40186192],
       [ 7490.95117552],
       [ 2695.53351128],
       [36660.93416497],
       [12263.53174794],
       [12449.45860984],
       [16348.84720323],
       [13657.9785808 ],
       [33286.39105261],
       [12911.51575765],
       [11470.50920976],


In [19]:
# Oцениваем качество, при помощи метода mean_squared_error для тестовой выборки ($ 5 9807, 05)
mean_squared_error(y_test, lr.predict(X_test))

33721836.07006891

In [20]:
# Oцениваем качество, при помощи метода mean_squared_error для выборки обучения ($ 6 110, 25)
mean_squared_error(y_train, lr.predict(X_train))

37335183.14645383

## Делаем предсказание для одного человека

In [21]:
# Заполняем данные по конкретному человеку. Например:
data = [{
    "age": 22,
    "sex": 1,
    "bmi": 31.350,
    "children": 1,
    "smoker": 0,
    "region_northeast": 0,
    "region_northwest": 1,
    "region_southeast": 0,
    "region_southwest": 0
}]

In [22]:
df_person = pd.DataFrame(data)
df_person.head()

,age,sex,bmi,children,smoker,region_northeast,region_northwest,region_southeast,region_southwest
0,22,1,31.35,1,0,0,1,0,0


In [ ]:
# Получаем предсказание по конкретному человеку $ 4301,99
lr.predict(df_person)

array([[4301.98713369]])

## Проведем сравнение с другими моделями - посмотрим какие модели лучше отработают на данных

In [25]:
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor

In [26]:
X_compare = df.drop("charges", axis=1)
y_compare = df["charges"]

# Делим данные
X_train_compare, X_test_compare, y_train_compare, y_test_compare = train_test_split(X_compare, y_compare, test_size=0.2, random_state=42)

In [33]:
# Создадим словарь моделей
models = {
    "Linear Regression": LinearRegression(),
    "Random Forest": RandomForestRegressor(n_estimators=200, random_state=42),
    "Gradient Boosting": GradientBoostingRegressor(n_estimators=200, random_state=42),
    "XGBoost": XGBRegressor(n_estimators=200, random_state=42, max_depth=3, learning_rate=0.05)
}

In [34]:
# Проведем обучение и оценку
results = {}
for name, model in models.items():
    model.fit(X_train, y_train.values.ravel())
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    results[name] = {"MSE": mse, "RMSE": rmse}

In [35]:
# Результаты
import pandas as pd
pd.DataFrame(results).T

,MSE,RMSE
Linear Regression,3.372184e+07,5807.050548
Random Forest,2.300562e+07,4796.417444
Gradient Boosting,2.084718e+07,4565.870707
XGBoost,1.945289e+07,4410.542824
